In [2]:
%load_ext autoreload
%autoreload 2
import torch
from model import *
from train import *

from test import test
import string
from utils import *
torch.manual_seed(1337)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
src_path = "./resources/news-commentary-v9.fr-en.en"
dst_path = "./resources/news-commentary-v9.fr-en.fr"
N = 3
n_embeddings = 32
n_head = 4
device = "cpu"
block_size = 50
start_token,end_token = "[","]"

In [4]:
src,src_vocab,dst,dst_vocab,chars = prepare_data(
    src_path,
    dst_path,
    50,
    add_start_end_tokens=True,
    num_lines=1000,
    start_token=start_token,
    end_token=end_token,
    remove_lines_containing_tokens=True
)
encoder_vocab_size = len(src_vocab)
decoder_vocab_size = len(dst_vocab)
encoder_vocab_size,decoder_vocab_size,len(chars)

(68, 79, 161290)

In [5]:
# for input,target,next_char in zip(decode_all(src,src_vocab),decode_all(dst,dst_vocab),decode(chars,dst_vocab)):
#     print(input,"   ",target,"   ---->   ",next_char)

In [6]:
model = AttentiveTranslator(
    N,
    encoder_vocab_size,
    decoder_vocab_size,
    n_embeddings,
    n_head,
    src.shape,
    dst.shape,
    device=device
).to(device=device)

In [7]:
src_train,src_valid,dst_train,dst_valid,chars_train,chars_valid = split(src,dst,chars)
len(src_train),len(src_valid),len(dst_train),len(dst_valid),len(chars_train),len(chars_valid)

(129032, 32258, 129032, 32258, 129032, 32258)

In [8]:
model = model.to(memory_format=torch.channels_last)
optim = torch.optim.Adam(
    model.parameters()
)
loss_fn = torch.nn.functional.cross_entropy
train_loader = arrays_to_loader(src_train,dst_train,chars_train,batch_size=32)
epochs=1
train(model,optim,loss_fn,epochs,train_loader,device,dst_vocab)

Loss: 2.2681405544281006: 100%|██████████| 4033/4033 [04:21<00:00, 15.40it/s]


In [9]:
from test import translate
translate(
    model,
    "hello there brother",
    block_size,
    src_vocab,
    dst_vocab,
    device,
    start_char=start_token,
    stop_char=end_token,
)

'le pars de pars de pars de pars de pars de pars de'